In [1]:
import logging
from lisa.conf import LisaLogging
LisaLogging.setup()

2018-09-19 16:14:34,146 INFO    : root         : Using LISA logging configuration:
2018-09-19 16:14:34,146 INFO    : root         :   /home/dourai01/Data/Git/lisa/py3k/logging.conf


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%pylab inline

import json
import os

# Support to access the remote target
import devlib
from lisa.env import TestEnv, TargetConfig

# Import support for Android devices
from lisa.android import Screen, Workload

# Support for trace events analysis
from lisa.trace import Trace

# Suport for FTrace events parsing and visualization
import trappy
import pandas as pd

from lisa.wlgen.rta import RTA, Periodic, Ramp
from time import sleep

from IPython.display import display

Populating the interactive namespace from numpy and matplotlib


In [4]:
linux_hikey = {
    "platform" : "linux",
    "board" : "hikey",
    "host": "pwrsft-hikey620-1",
    #"username" : "root",
    #"password" : "root",
    "modules" : ["sched", "cgroups", "hotplug"],
    "tools" : ["taskset", "rt-app"],
    "rtapp-calib" :  {
        "0": 302, "1": 302, "2": 302, "3": 302, "4": 136, "5": 136, "6": 136, "7": 136
     }
}

In [5]:
linux_juno = {
    "platform" : "linux",
    "board" : "juno",
    "host": "192.168.0.1",
    "username" : "root",
    "password" : "root",
    "modules" : ["sched", "cgroups", "hotplug"],
    "tools" : ["taskset", "rt-app"]
}

In [6]:
android_hikey = {
    "platform" : "android",
    "board" : "hikey960",
    "modules" : ["sched", "hotplug"],
    "tools" : ["taskset", "rt-app"]
}

In [ ]:
te = TestEnv(TargetConfig(linux_hikey))
target = te.target

### Generics

In [9]:
from lisa.tests.kernel.scheduler.eas_behaviour import *

In [10]:
tests = [OneSmallTask]#, ThreeSmallTasks, TwoBigTasks, TwoBigThreeSmall, RampUp, RampDown, EnergyModelWakeMigration]

In [11]:
res = {}

for test in tests:
    bundle = test.from_target(te)
    res[test.__name__] = bundle

2018-09-19 16:26:29,611 INFO    : TestEnv      : Using configuration provided RTApp calibration
2018-09-19 16:26:29,611 INFO    : TestEnv      : Using RT-App calibration values:
2018-09-19 16:26:29,612 INFO    : TestEnv      :    {"0": 302, "1": 302, "2": 302, "3": 302, "4": 136, "5": 136, "6": 136, "7": 136}
2018-09-19 16:26:30,011 INFO    : RTA          : Calibration value: 136
2018-09-19 16:26:30,012 INFO    : RTA          : Default policy: SCHED_OTHER
2018-09-19 16:26:30,012 INFO    : RTA          : ------------------------
2018-09-19 16:26:30,013 INFO    : RTA          : task [small], sched: using default policy
2018-09-19 16:26:30,014 INFO    : RTA          :  | start delay: 0.000000 [s]
2018-09-19 16:26:30,014 INFO    : RTA          :  | loops count: 1
2018-09-19 16:26:30,015 INFO    : RTA          :  + phase_000001
2018-09-19 16:26:30,016 INFO    : Phase        :  | duration 1.000000 [s] (62 loops)
2018-09-19 16:26:30,016 INFO    : Phase        :  |  period    16000 [us], duty_

ConstructorError: found unconstructable recursive node
  in "/home/dourai01/Data/Git/lisa/py3k/results/20180919_162625/OneSmallTask.yaml", line 5, column 5

In [ ]:
for test_name, bundle in res.items():
    print(test_name)
    print("---")
    print("test_slack: {}".format(bundle.test_slack())
    print("test_task_placement: {}".format(bundle.test_task_placement())
    print()

### CPU sanity check

In [ ]:
from lisa.tests.kernel.misc.capacity_sanity import CapacitySanityCheck

In [ ]:
bundle = CapacitySanityCheck.from_target(te)

In [ ]:
print(bundle.test_capacity_sanity())

### Hotplug

In [ ]:
from lisa.tests.kernel.hotplug.torture import HotplugTorture

In [ ]:
bundle = HotplugTorture.from_target(te)

In [ ]:
bundle.test_cpus_alive().result

In [ ]:
bundle.test_cpus_alive().metrics

In [ ]:
bundle.test_target_alive().result